In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
!nvidia-smi

Wed Dec 14 10:22:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    33W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
import os
os.chdir("/content/drive/MyDrive")

In [6]:
from shutil import copy
from collections import defaultdict
import scipy
import numpy as np
import matplotlib.pyplot as plt

import random
import cv2
import glob
from PIL import Image, ImageEnhance
import PIL.ImageOps

import torch
import torchvision
from torch import nn


import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

import tensorflow as tf
from tensorflow import keras
from tensorflow.python.ops.clip_ops import clip_by_global_norm
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers
import tensorflow_datasets as tfds


# Theano(th)와 Tensorflow(tf) 모두와 호환이 되는 Keras 모듈을 작성
import keras.backend as K
from keras import regularizers
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, BatchNormalization, Dropout, Flatten, Dense, concatenate
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator # 데이터 전처리
from keras.callbacks import ModelCheckpoint, CSVLogger
#from keras.optimizers import SGD

from keras_preprocessing.image import array_to_img, img_to_array, load_img

from sklearn.model_selection import train_test_split


from deeplab2.model.pixel_encoder import moat
from adabelief_tf import AdaBeliefOptimizer

### https://www.tensorflow.org/datasets/catalog/cifar100

In [7]:
# 'cifar100' 대신 'food101' 이나 다른 데이터셋 이름 넣으면 ok
from tensorflow.keras.datasets import cifar100

(train_images, train_labels), (test_images, test_labels) = cifar100.load_data()

tr_images, val_images, tr_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.1, random_state=34)
tr1_images, tr2_images, tr1_labels, tr2_labels = train_test_split(tr_images, tr_labels, test_size=0.5, shuffle=False)

nb_train_samples =  tr_images.shape[0]
nb_train1_samples =  tr1_images.shape[0]
nb_train2_samples =  tr2_images.shape[0] 
nb_validation_samples = val_images.shape[0]
nb_test_samples = test_images.shape[0]
batch_size = 32

print('Train:', tr_images.shape, tr_labels.shape)
print('Train1:', tr1_images.shape, tr1_labels.shape)
print('Train2:', tr2_images.shape, tr2_labels.shape)
print('Valid:', val_images.shape, val_labels.shape)
print('Test:', test_images.shape, test_labels.shape)

Train: (45000, 32, 32, 3) (45000, 1)
Train1: (22500, 32, 32, 3) (22500, 1)
Train2: (22500, 32, 32, 3) (22500, 1)
Valid: (5000, 32, 32, 3) (5000, 1)
Test: (10000, 32, 32, 3) (10000, 1)


In [8]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,        # Shear angle in counter-clockwise direction as radians
    zoom_range=0.3,         # Range for random zoom. If a float
    horizontal_flip=True,   # Randomly flip inputs horizontally
    fill_mode='nearest') 

train1_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,        # Shear angle in counter-clockwise direction as radians
    zoom_range=0.3,         # Range for random zoom. If a float
    horizontal_flip=True,   # Randomly flip inputs horizontally
    fill_mode='nearest') 

train2_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,        # Shear angle in counter-clockwise direction as radians
    zoom_range=0.3,         # Range for random zoom. If a float
    horizontal_flip=True,   # Randomly flip inputs horizontally
    fill_mode='nearest') 

val_datagen = ImageDataGenerator(
    rescale=1./255,)

test_datagen = ImageDataGenerator(
    rescale=1./255,)

In [9]:
# Numpy Array Iterator 객체 생성하여 모델 인풋으로 배치 사이즈만큼 집어넣을 준비
train_generator = train_datagen.flow(x=tr_images, y=tr_labels, batch_size=32, shuffle=True)
train1_generator = train1_datagen.flow(x=tr1_images, y=tr1_labels, batch_size=32, shuffle=True)
train2_generator = train2_datagen.flow(x=tr2_images, y=tr2_labels, batch_size=32, shuffle=True)

validation_generator = val_datagen.flow(x=val_images, y=val_labels, batch_size=32, shuffle=False)

test_generator = test_datagen.flow(x=test_images, y=test_labels, batch_size=32, shuffle=False)

In [10]:
moat1 = moat.get_model(name='tiny_moat1_pretrain_256_no_pe_1k',input_shape=[32,32,3])
moat1 = moat._load_moat_pretrained_checkpoint(moat1, path="./model-ckpt-0")

o1 = moat1.output["stage5"]
r1 = tf.keras.layers.Flatten(name='flatten_layer1')(o1)
r1 = tf.keras.layers.BatchNormalization()(r1)
out1 = tf.keras.layers.Dense(
    units=100, activation='softmax', name='output_layer', kernel_initializer="he_normal")(r1)  

def create_model():
    return tf.keras.models.Model(moat1.input, out1)

In [11]:
model1 = create_model()
#model1.load_weights('cifar_final_disjoint1_1.hdf5')

In [12]:
import warnings
warnings.filterwarnings('ignore')

opt = AdaBeliefOptimizer(learning_rate=1e-5, beta_1= 0.9, beta_2=0.999, weight_decay= 0.05,  epsilon=1e-14)

model1.compile(optimizer=opt,
          loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())

checkpointer = ModelCheckpoint(filepath='cifar_final_disjoint1_1.hdf5', verbose=1, monitor ="val_sparse_categorical_accuracy",save_best_only=True)
csv_logger = CSVLogger('cifar_final_disjoint1_1.log')

history = model1.fit(train1_generator,
                steps_per_epoch = nb_train1_samples // batch_size,      
                validation_data=validation_generator,
                validation_steps=nb_validation_samples // batch_size,  
                epochs=80,                                             
                verbose=1,
                callbacks=[csv_logger, checkpointer])

Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
>=0.1.0 (Current 0.2.1)  1e-14  supported          default: True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended epsilon = 1e-7                                  Recommended epsilon = 1e-14
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.

Epoch 1/80
703/703 [==============================] - ETA: 0s - loss: 5.4963 - sparse_categorical